In [1]:
import h5py
import numpy as np

file_path = r"D:\BTP\sub-1_ses-1_task-bcimici_meg.mat"

with h5py.File(file_path, 'r') as f:
    data = f['dataMAT']
    trial_refs = data['trial']
    trial_info_list = data['trialinfo'][:]
    
    trials_list = []
    trial_info_list = []
    
    for i in range(200):
        trial_ref = trial_refs[i,0]
        trial_data = f[trial_ref][2001:7001]  # (channels, timepoints)
        trials_list.append(trial_data)
        

# Convert lists to arrays
trials_array = np.array(trials_list)
trial_info_array = np.array(trial_info_list)
print(trial_info_array.shape)

# Save to .npy files
np.save(r"meg_trials.npy", trials_array)
np.save(r"meg_trial_info.npy", trial_info_array)


(0,)


In [6]:
import h5py
import numpy as np

file_path = r"D:\BTP\sub-1_ses-1_task-bcimici_meg.mat"

with h5py.File(file_path, 'r') as f:
    data = f['dataMAT']
    trial_refs = data['trial']
    trial_info_array = data['trialinfo'][:]  # Directly load trialinfo
    
    trials_list = []
    
    for i in range(200):
        trial_ref = trial_refs[i,0]
        trial_data = f[trial_ref][2001:7001]  # (channels, timepoints)
        trials_list.append(trial_data)

# Convert trials list to array
trials_array = np.array(trials_list)
print(trial_info_array.shape)  # Should now show (200, 1)

# Save to .npy files
np.save(r"meg_trials.npy", trials_array)
np.save(r"meg_trial_info.npy", trial_info_array)


(1, 200)


In [7]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.nn as nn 
import torch.optim as optim 

class EEGDataset(Dataset):
    def __init__(self, trials, labels):
        # Trials: (num_trials, 750, 22)
        # Labels: (num_trials,)
        
        # Normalize data per channel
        self.data = torch.tensor(trials, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)
        
        # Map original labels to 0-3
        self.label_mapping = {1: 0, 2: 1, 3: 2, 4: 3}
        self.labels = torch.tensor([self.label_mapping[int(x)] for x in labels])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Assuming you have loaded your data into trials and labels arrays
trials = np.load('meg_trials.npy')  # Shape (288, 750, 22)
labels = np.load('meg_trial_info.npy')     # Shape (288,)
print(trials.shape)
print(labels.shape)
labels = labels.reshape(200, 1)
print(labels.shape)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    trials, labels, test_size=0.2, stratify=labels, random_state=42
)

# Create datasets and dataloaders
train_dataset = EEGDataset(X_train, y_train)
test_dataset = EEGDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

(200, 4999, 306)
(1, 200)
(200, 1)


C:\Users\ASUS\AppData\Local\Temp\ipykernel_22080\30888903.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.labels = torch.tensor([self.label_mapping[int(x)] for x in labels])


In [8]:


class LTC_Cell(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LTC_Cell, self).__init__()
        self.hidden_dim = hidden_dim
        
        # Initialize layers with proper device awareness
        self.W_xh = nn.Linear(input_dim, hidden_dim)
        self.W_hh = nn.Linear(hidden_dim, hidden_dim, bias=False)
        self.W_tau = nn.Linear(hidden_dim, hidden_dim)
        
    def forward(self, x, h):
        # Ensure operations stay on same device
        tau = torch.sigmoid(self.W_tau(h)) + 0.1
        dh = -h / tau + torch.tanh(self.W_xh(x) + self.W_hh(h))
        return h + 0.1 * dh
        
class EnhancedLTC(nn.Module):
    def __init__(self, input_dim=22, hidden_dim=64, output_dim=4, num_layers=2):
        super(EnhancedLTC, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        
        self.ltc_layers = nn.ModuleList([
            LTC_Cell(input_dim if i==0 else hidden_dim, hidden_dim) 
            for i in range(num_layers)
        ])
        
        self.attention = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.Tanh(),
            nn.Linear(hidden_dim, 1)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, 32),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        batch_size, seq_len, _ = x.size()
        
        # Initialize hidden states on same device as input
        hiddens = [torch.zeros(batch_size, self.hidden_dim, device=x.device) 
                  for _ in range(self.num_layers)]
        
        all_hidden = []
        
        for t in range(seq_len):
            x_t = x[:, t, :]
            for layer_idx in range(self.num_layers):
                hiddens[layer_idx] = self.ltc_layers[layer_idx](
                    x_t if layer_idx == 0 else hiddens[layer_idx-1],
                    hiddens[layer_idx]
                )
            all_hidden.append(hiddens[-1])
        
        hidden_stack = torch.stack(all_hidden, dim=1) # it is converting 2d list tensor to 3d for each batch
        attn_weights = torch.softmax(self.attention(hidden_stack), dim=1)
        context = torch.sum(attn_weights * hidden_stack, dim=1)
        
        return self.classifier(context)

In [9]:
# Initialize model
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EnhancedLTC(input_dim=306, hidden_dim=128, output_dim=4, num_layers=2).to(device)
model_path = 'meg2_ltc_model.pth'
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path))
    print("Loaded model from checkpoint.")
else:
    print("No checkpoint found. Starting from scratch.")

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

num_epochs = 100
previous_val_acc = None
same_acc_streak = 0
best_val_acc = 0
val_acc_list = []

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    
    for batch, (data, labels) in enumerate(train_loader):
        # Move data to device
        data, labels = data.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(data)
        loss = criterion(outputs, labels)
        loss.backward()
        
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss, correct, total = 0, 0, 0
   
    
    with torch.no_grad():
        for data, labels in test_loader:
            data, labels = data.to(device), labels.to(device)
            outputs = model(data)
            val_loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    val_acc = 100 * correct / total
    train_loss /= len(train_loader)
    val_loss /= len(test_loader)
    scheduler.step(val_loss)

    if previous_val_acc is None or val_acc != previous_val_acc:
        same_acc_streak = 0  # Reset counter if there's any change in accuracy
    else:
        same_acc_streak += 1  # Increment counter if accuracy is the same

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'meg2_ltc_model.pth')

    val_acc_list.append(val_acc)
    
    print(f'Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | 'f'Same accuracy streak: {same_acc_streak}/10')

    # Early stopping triggered after 10 consecutive epochs with no change in accuracy
    if same_acc_streak >= 10:
        print(f'\nEarly stopping triggered after {epoch+1} epochs!')
        break
    
    # Update previous_val_acc for the next iteration
    previous_val_acc = val_acc

# Load best model and final evaluation
avg_val_acc = sum(val_acc_list) / len(val_acc_list)
print(f'\nTraining complete. Best validation accuracy: {best_val_acc:.2f}%')
print(f'Average validation accuracy over {len(val_acc_list)} epochs: {avg_val_acc:.2f}%')



C:\Users\ASUS\AppData\Local\Temp\ipykernel_22080\3071847531.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Loaded model from checkpoint.
Epoch 1/100 | Train Loss: 1.3898 | Val Loss: 1.3889 | Val Acc: 25.00% | Same accuracy streak: 0/10


KeyboardInterrupt: 